# 01_EDA

## Imports

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

## Read data

In [31]:
raw_df = pd.read_csv('../data/raw/dataset_10k_final.csv')
windowed_df = pd.read_csv('../data/processed/windowed_dataset.csv')
print("Raw dataset shape:", raw_df.shape)
print("Windowed dataset shape:", windowed_df.shape) 


Raw dataset shape: (9277, 33)
Windowed dataset shape: (24005, 94)


In [32]:
prop_raw = raw_df['is_attack'].value_counts(normalize=True).reindex([0,1], fill_value=0)
prop_win = windowed_df['is_attack'].value_counts(normalize=True).reindex([0,1], fill_value=0)

datasets = ['Raw Dataset', 'Windowed Dataset']
normal_vals = [prop_raw[0], prop_win[0]]
attack_vals = [prop_raw[1], prop_win[1]]

fig = go.Figure()
fig.add_bar(x=datasets, y=normal_vals, name='Normal')
fig.add_bar(x=datasets, y=attack_vals, name='Ataque')

fig.update_layout(
    title="Proporción de ataques en los datasets",
    barmode='group',
    yaxis=dict(range=[0,1])
)
fig.write_image(
    "../reports/figures/dataset_proportion.png",
    width=1000,      # ancho en píxeles
    height=600,      # alto en píxeles
    scale=2          # factor de escalado (2 = ~200 DPI)
)
fig.show()

## Null Values

In [33]:
import pandas as pd

# Número absoluto de nulos por columna
null_counts = windowed_df.isnull().sum()

# Porcentaje de nulos por columna
null_percent = windowed_df.isnull().mean() * 100

# Mostrar en un DataFrame ordenado
null_summary = pd.DataFrame({
    'nulos': null_counts,
    'porcentaje_nulos (%)': null_percent
}).sort_values(by='porcentaje_nulos (%)', ascending=False)

print(null_summary)

                                 nulos  porcentaje_nulos (%)
time_since_last_conn_std         21937             91.385128
bytes_ratio_std                  21936             91.380962
interval_stddev_std              21936             91.380962
conn_count_60s_std               21936             91.380962
ip_first_seen_hours_ago_std      21936             91.380962
conn_interval_std                21936             91.380962
ja3_frequency_std                21936             91.380962
conn_state_encoded_std           21936             91.380962
recon_pattern_score_std          21936             91.380962
duration_zscore_std              21936             91.380962
conn_count_300s_std              21936             91.380962
recent_activity_score_std        21936             91.380962
duration_std                     21936             91.380962
unique_ja3_from_ip_std           21936             91.380962
is_known_ip_std                  21936             91.380962
recent_docker_event_std 